In [ ]:
# AnxLight Launcher v0.0.1 - 2025-06-19
# @title Setup AnxLight Environment and Launch Gradio UI
import os
import subprocess
import runpy
import sys
from pathlib import Path

# --- Configuration ---
REPO_URL = "https://github.com/drf0rk/AnxLight.git"
REPO_DIR_NAME = "AnxLight"
BRANCH_NAME = "feature/backend-integration"
APP_SCRIPT_RELATIVE_PATH = "scripts/main_gradio_app.py"

# --- 1. Clone/Update Repository ---
print("--- Ensuring latest repository code from branch:", BRANCH_NAME, "---")
original_cwd = os.getcwd()

repo_path = Path(original_cwd) / REPO_DIR_NAME

if not repo_path.exists():
    print(f"Cloning repository '{REPO_URL}' (branch: {BRANCH_NAME}) into '{repo_path}'...")
    subprocess.run(["git", "clone", "-b", BRANCH_NAME, REPO_URL, str(repo_path)], check=True)
    os.chdir(str(repo_path))
else:
    print(f"Directory '{repo_path}' already exists.")
    os.chdir(str(repo_path))
    print(f"Fetching updates and switching to branch '{BRANCH_NAME}' in '{os.getcwd()}'...")
    try:
        subprocess.run(["git", "fetch"], check=True)
        subprocess.run(["git", "checkout", "-B", BRANCH_NAME, f"origin/{BRANCH_NAME}"], check=True)
        subprocess.run(["git", "pull", "origin", BRANCH_NAME], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error during git operations (fetch/checkout/pull): {e}")
        print("Please ensure the branch exists and you have connectivity.")

# --- 2. Install Dependencies ---
print("\n--- Installing Dependencies ---")
print("Installing Gradio...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "gradio"], check=True, capture_output=False, text=False)

# --- 3. Setup Environment Variables & Run Application ---
print(f"\n--- Launching Gradio Application ---")
PROJECT_ROOT = os.getcwd()
print(f"Current Project Root (cwd for main_gradio_app.py): {PROJECT_ROOT}")

try:
    os.environ['IS_COLAB'] = 'true'
    os.environ['home_path'] = PROJECT_ROOT 
    os.environ['scr_path'] = PROJECT_ROOT 
    os.environ['settings_path'] = os.path.join(PROJECT_ROOT, 'anxlight_config.json')
    os.environ['venv_path'] = os.path.join(PROJECT_ROOT, 'venv')
    existing_pythonpath = os.environ.get('PYTHONPATH', '')
    anxlight_modules_path = PROJECT_ROOT
    os.environ['PYTHONPATH'] = f"{anxlight_modules_path}{os.pathsep}{existing_pythonpath}" if existing_pythonpath else anxlight_modules_path

    print(f"[Notebook] Set environment variable home_path to: {os.environ['home_path']}")
    print(f"[Notebook] Set environment variable scr_path to: {os.environ['scr_path']}")
    print(f"[Notebook] Set environment variable settings_path to: {os.environ['settings_path']}")
    print(f"[Notebook] Set environment variable venv_path to: {os.environ['venv_path']}")
    print(f"[Notebook] Updated PYTHONPATH to: {os.environ['PYTHONPATH']}")

    print(f"Attempting to launch '{APP_SCRIPT_RELATIVE_PATH}' from '{PROJECT_ROOT}' using runpy...")
    runpy.run_path(APP_SCRIPT_RELATIVE_PATH, run_name="__main__")
except ModuleNotFoundError as e:
    print(f"Error: A module was not found. Details: {e}")
    print(f"Current sys.path: {sys.path}")
except FileNotFoundError:
    print(f"Error: The script '{APP_SCRIPT_RELATIVE_PATH}' was not found in '{PROJECT_ROOT}'.")
except Exception as e:
    print(f"An error occurred: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()
finally:
    os.chdir(original_cwd)
    print(f"\nChanged working directory back to: {os.getcwd()}")